In [1]:
import numpy as np
from numpy import convolve
from numpy import genfromtxt
from scipy.interpolate import interp1d
from scipy.signal import savgol_filter
from scipy.integrate import RK45
from bokeh.plotting import figure, output_notebook, show
from bokeh.layouts import row, column
from bokeh.io import push_notebook, curdoc, export_png
from bokeh.models import Range1d, Label, Span, LinearAxis
from bokeh.models.glyphs import Line
from bokeh.themes import Theme
from bokeh.models.tickers import FixedTicker
output_notebook()
import ipywidgets as widgets
# import pandas as pd
# import qgrid
import ipywidgets
from ipywidgets import IntProgress, HBox, VBox, GridBox, Layout, Tab, FloatText
import sys
sys.path.insert(0,'C:\\Users\\Alan\\Anaconda3\\lib\\site-packages')

Loading BokehJS ...

Parameters Determined by Fiber Geometry and Materials

In [2]:
style = {'description_width': 'initial'}
box_titles = ['Core Diameter (micron)', 'Cladding Diameter (micron)', 'Fiber Loss (dB/m)',
            'Wavelength-CW Pump (nm)', 'Power - CW Pump (W)',
            'Pulse Energy - Fund (mJ)', 'Pulse Width - Fund (ns)', 'Wavelength - Fund (nm)', 'Pulse Width - Seed (ps)',
            'Pulse Energy - Seed (micJ)', 'Raman Shift (cm^-1)', 'Raman Gain (1E-12 cm/W)',
            'Raman Gain Width (cm^-1)', '2nd Stokes Loss (dB/m)', 'Lumped 2nd Stokes Loss (dB)','Fiber Length (meter)',
            'Refractive Index', 'Time Increment (psec)', 'Pump Loss (dB/m)', '# of Regen Passes']
box_values = [25.,250.,0.,
            976., 100., 40, 15,
            2040., 100, 10,
            450, 5,
            300, 0.0, 0, 0.2,
            1.5, 1.,4.0, 1]
box_steps = [1., 1., 0.1,
             10, 1, 10, 1,
             10, 10, 0.1,
             5, 1,
             5, 1, 0.1, 0.1,
             0.1, 1, 0.1, 1]
items = [FloatText(value=box_values[i], description=box_titles[i], step=box_steps[i], style=style) for i in range (len(box_titles))]
for i in range (len(items)): items[i].layout.width='230px'
gb=GridBox(items, layout=Layout(grid_template_columns="repeat(3, 250px)"))
gb

GridBox(children=(FloatText(value=25.0, description='Core Diameter (micron)', layout=Layout(width='230px'), st…

In [3]:
# Assign Values
coreD = gb.children[0].value*1e-4 # Core diameter in cm
cladD = gb.children[1].value*1e-4 # Cladding diameter in cm
fiber_loss = gb.children[2].value/100/4.34 # fiber background loss in inverse cm
pump_wavelength = gb.children[3].value # pump wavelenth in nm
pump_power = gb.children[4].value # pump power in W
f_energy = gb.children[5].value*1e-3 # Energy of fundamenatl pulse in mJ
f_width = gb.children[6].value*1e-9 # FWHM of fundamenatl pulse in nS
fundamental_wavelength = gb.children[7].value # wavelength of fundamenatl in nm
FWHM1 = gb.children[8].value*1e-12 # FWHM of Stokes seed in sec
Es10 = gb.children[9].value*1e-6 # Stokes seed energy in J
rshift = gb.children[10].value # Raman shift in inverse cm
raman_gain = gb.children[11].value*1e-12 # Peak Raman Gain in cm/W
raman_width = gb.children[12].value # Raman Gain Width in inverse cm
stokes2_loss = gb.children[13].value/4.34/1e2 # 2nd Stokes Loss in inverse cm
stokes3_loss = 140
stokes4_loss = 140
stokes2_loss_lump = 10**(-gb.children[14].value/10.0) # Lumped 2nd Stokes loss
L = gb.children[15].value # Fiber length in m
nref = gb.children[16].value # Refractive index
delta_t = gb.children[17].value*1e-12  # Time increment in sec
pump_loss0 = gb.children[18].value/4.34/100  # Pump loss in inverse cm
Npass = int(gb.children[19].value) # Number of single passes in regenerative amplifier


# Print Key Values
print('Fundamental Wavelength (nm) = ','{:5.0f}'.format(fundamental_wavelength))
stokesw1 = 1e7/(-rshift+1e7/fundamental_wavelength) # Wavelength of 1st Stokes signal in nm
print('First Stokes Wavelength (nm) = ','{:5.0f}'.format(stokesw1))
stokesw2 = 1e7/(-rshift+1e7/stokesw1) # Wavelength of 2st Stokes signal in nm
print('Second Stokes Wavelength (nm) = ','{:5.0f}'.format(stokesw2))
stokesw3 = 1e7/(-rshift+1e7/stokesw2) # Wavelength of 3rd Stokes signal in nm
print('Third Stokes Wavelength (nm) = ','{:5.0f}'.format(stokesw3))
stokesw4 = 1e7/(-rshift+1e7/stokesw3) # Wavelength of 4th Stokes signal in nm
print('Fourth Stokes Wavelength (nm) = ','{:5.0f}'.format(stokesw3))

Aeff = np.pi*(coreD/2)**2
raman_width_Hz = raman_width/(1e7/stokesw1)*(3e17/stokesw1) # Width of equivalent spontaneous Raman input in Hz

Fundamental Wavelength (nm) =   2040
First Stokes Wavelength (nm) =   2246
Second Stokes Wavelength (nm) =   2499
Third Stokes Wavelength (nm) =   2815
Fourth Stokes Wavelength (nm) =   2815


In [4]:
# Create Signals
c=2.99792458e8
vg = c/nref
delta_L = vg * delta_t

hnu_stokes1=1239.842/stokesw1*1.6022e-19
hnu_stokes2=1239.842/stokesw2*1.6022e-19
hnu_stokes3=1239.842/stokesw3*1.6022e-19
hnu_stokes4=1239.842/stokesw3*1.6022e-19

# Create Signals
MM=1.3
N=int(round((2*MM+1)*L/delta_L))+1
t=np.linspace(-MM*L/vg, (MM+1)*L/vg,N)
z=np.linspace(-MM*L,(MM+1)*L,N)
# tend=(MM+0.5)*L/vg
tend1=0
tend2=2*L/vg
tend3=L/vg
fund_forw=(f_energy/Aeff)*(np.sqrt(4*np.log(2)/np.pi)/f_width)*np.exp(-4*np.log(2)*((t-tend1)/(f_width))**2)*2
fund_back=(f_energy/Aeff)*(np.sqrt(4*np.log(2)/np.pi)/f_width)*np.exp(-4*np.log(2)*((t-tend2)/(f_width))**2)*2
I01=(Es10/Aeff)*(np.sqrt(4*np.log(2)/np.pi)/FWHM1)
stokes1 = I01*np.exp(-4*np.log(2)*(t/(FWHM1))**2)+hnu_stokes1*raman_width_Hz/Aeff
# +hnu_stokes1*raman_width_Hz/Aeff
stokes2=np.zeros(N)
stokes3=np.zeros(N)
stokes4=np.zeros(N)
convsignal = np.ones(1)/1
int_mask = np.ones(N)

In [5]:
# Setup Figure
pg=figure(plot_width=650, plot_height=400, y_axis_type="log")
pg.min_border_top = 10

label1=Label(x=70, y=215, x_units='screen', y_units='screen', text='1st Stokes @ Output',
             text_font_size='10pt', text_color='darkblue')
label1a=Label(x=70, y=195, x_units='screen', y_units='screen', text='',
                 text_font_size='10pt', text_color='darkblue')
label2=Label(x=70, y=175, x_units='screen', y_units='screen', text='2nd Stokes @ Output',
             text_font_size='10pt', text_color='darkblue')
label2a=Label(x=70, y=155, x_units='screen', y_units='screen', text='',
                 text_font_size='10pt', text_color='darkblue')
label3=Label(x=70, y=135, x_units='screen', y_units='screen', text='3rd Stokes @ Output',
             text_font_size='10pt', text_color='darkblue')
label3a=Label(x=70, y=115, x_units='screen', y_units='screen', text='',
                 text_font_size='10pt', text_color='darkblue')
label4=Label(x=70, y=95, x_units='screen', y_units='screen', text='4th Stokes @ Output',
             text_font_size='10pt', text_color='darkblue')
label4a=Label(x=70, y=75, x_units='screen', y_units='screen', text='Extraction Efficiecy',
             text_font_size='10pt', text_color='darkblue')

pg.x_range = Range1d(0,L)
pg.y_range = Range1d(1e-10,7e7)
cff=pg.line(z, fund_forw*Aeff, legend_label="Forward Fundamental",
           line_width=2, color='blue')
cfb=pg.line(z, fund_back*Aeff, legend_label="Backward Fundamental",
            line_width=2, line_dash='dashed', color='blue')
c1=pg.line(z, stokes1*Aeff, legend_label="Seed",
          line_width=2, color='red')
c2=pg.line(z, stokes2*Aeff, legend_label="2nd Stokes", color='red')
c3=pg.line(z, stokes3*Aeff, legend_label="3rd Stokes", color='black')
c4=pg.line(z, stokes4*Aeff, legend_label="4th Stokes", color='yellow')
# numpy_signal1 = np.asarray(signal1)
# smooth_signal1 = savgol_filter(numpy_signal1,7,3)
pg.legend.location = "top_left"; pg.legend.background_fill_alpha = 0.3
pg.xaxis.axis_label = 'Position (meter)'; pg.xaxis.axis_label_text_font_size = "12pt"
pg.xaxis.major_label_text_font_size = "12pt"
pg.yaxis.axis_label = 'Signal (Watt)'; pg.yaxis.axis_label_text_font_size = "12pt"
pg.yaxis.major_label_text_font_size = "12pt"

show(pg)

In [ ]:
# Create Signals
MM=1.3
N=int(round((2*MM+1)*L/delta_L))+1
t=np.linspace(-MM*L/vg, (MM+1)*L/vg,N)
z=np.linspace(-MM*L,(MM+1)*L,N)
# tend=(MM+0.5)*L/vg
tend1=0
tend2=2*L/vg
tend3=L/vg
fund_forw=(f_energy/Aeff)*(np.sqrt(4*np.log(2)/np.pi)/f_width)*np.exp(-4*np.log(2)*((t-tend1)/(f_width))**2)*2
fund_back=(f_energy/Aeff)*(np.sqrt(4*np.log(2)/np.pi)/f_width)*np.exp(-4*np.log(2)*((t-tend2)/(f_width))**2)*2
I01=(Es10/Aeff)*(np.sqrt(4*np.log(2)/np.pi)/FWHM1)
stokes1 = I01*np.exp(-4*np.log(2)*(t/(FWHM1))**2)+hnu_stokes1*raman_width_Hz/Aeff
stokes2=np.zeros(N)
stokes3=np.zeros(N)
stokes4=np.zeros(N)
convsignal = np.ones(1)/1

int_mask = np.zeros(N); int_mask[int(round(N/3)):int(round(2*N/3))]=1

nh=show(pg,notebook_handle='true');

import time
t0 = time.time()

M=1
delta_L=M*delta_L
P = IntProgress(min=0, max=int(round(N/(2*MM+1)/M)), description='Progress', bar_style='success'); display(P)
P2 = IntProgress(min=0, max=Npass, description='Progress', bar_style='success'); display(P2)

Eps = np.zeros([4,Npass+1])
Eps[0,0] = Es10*1e6

fundamental=fund_forw
signal1=stokes1
signal2=stokes2
signal3=stokes3
signal4=stokes4
hnu_signal1=hnu_stokes1
hnu_signal2=hnu_stokes2
hnu_signal3=hnu_stokes3
hnu_signal4=hnu_stokes4
def gaussian(x, mu, sig):
    return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))
convsignal=gaussian(t,0,FWHM1)/np.sum(gaussian(t,0,FWHM1))

for k in range(Npass):
    if k % 2 == 0:
        fact=1
    else:
        fact=-1
    for i in range(int(round(N/(2*MM+1)/M))):
        fund_back[int(round((MM+1)*L/delta_L))]=fundamental[int(round((MM+1)*L/delta_L))]
        fk1 = (-fundamental*signal1*raman_gain*stokesw1/fundamental_wavelength
                              - fundamental*fiber_loss)*delta_L
        fbk1 = (-fund_back*signal1*raman_gain*stokesw1/fundamental_wavelength
                              - fund_back*fiber_loss)*delta_L
        s1k1 = ((fundamental+fund_back)*signal1*raman_gain-signal1*signal2*raman_gain*stokesw2/stokesw1-signal1*fiber_loss)*delta_L
        s2k1 = (signal1*signal2*raman_gain + signal1*raman_gain*hnu_signal2*raman_width_Hz/Aeff
                        -signal2*signal3*raman_gain*stokesw3/stokesw2 - signal2*fiber_loss - signal2*stokes2_loss)*delta_L
        
        s3k1 = (signal2*signal3*raman_gain + signal2*raman_gain*hnu_signal3*raman_width_Hz/Aeff
                         - signal3*fiber_loss - signal3*stokes3_loss)*delta_L
        s4k1 = (signal3*signal4*raman_gain + signal3*raman_gain*hnu_signal4*raman_width_Hz/Aeff
                         - signal4*fiber_loss - signal4*stokes4_loss)*delta_L

        fk2 = (-(fundamental+fk1/2)*signal1*raman_gain*stokesw1/fundamental_wavelength
                              - (fundamental+fk1/2)*fiber_loss)*delta_L
        fbk2 = (-(fund_back+fbk1/2)*signal1*raman_gain*stokesw1/fundamental_wavelength
                              - (fund_back+fbk1/2)*fiber_loss)*delta_L
        s1k2 = ((fundamental+fund_back)*(signal1+s1k1/2)*raman_gain
                        -(signal1+s1k1/2)*signal2*raman_gain*stokesw2/stokesw1
                        -(signal1+s1k1/2)*fiber_loss)*delta_L
        s2k2 = (signal1*(signal2+s2k1/2)*raman_gain + signal1*raman_gain*hnu_signal2*raman_width_Hz/Aeff
                         - (signal2+s2k1/2)*fiber_loss - (signal2+s2k1/2)*stokes2_loss)*delta_L
        s3k2 = (signal2*(signal3+s3k1/2)*raman_gain + signal2*raman_gain*hnu_signal3*raman_width_Hz/Aeff
                         - (signal3+s3k1/2)*fiber_loss - (signal3+s3k1/2)*stokes3_loss)*delta_L
        s4k2 = (signal3*(signal4+s4k1/2)*raman_gain + signal3*raman_gain*hnu_signal4*raman_width_Hz/Aeff
                         - (signal4+s4k1/2)*fiber_loss - (signal4+s4k1/2)*stokes4_loss)*delta_L
        
        fk3 = (-(fundamental+fk2/2)*signal1*raman_gain*stokesw1/fundamental_wavelength
                              - (fundamental+fk2/2)*fiber_loss)*delta_L
        fbk3 = (-(fund_back+fbk2/2)*signal1*raman_gain*stokesw1/fundamental_wavelength
                              - (fundamental+fk2/2)*fiber_loss)*delta_L
        s1k3 = ((fundamental+fund_back)*(signal1+s1k2/2)*raman_gain
                        -(signal1+s1k2/2)*signal2*raman_gain*stokesw2/stokesw1
                        -(signal1+s1k2/2)*fiber_loss)*delta_L
        s2k3 = (signal1*(signal2+s2k2/2)*raman_gain + signal1*raman_gain*hnu_signal2*raman_width_Hz/Aeff
                         - (signal2+s2k2/2)*fiber_loss - (signal2+s2k2/2)*stokes2_loss)*delta_L
        s3k3 = (signal2*(signal3+s3k2/2)*raman_gain + signal2*raman_gain*hnu_signal3*raman_width_Hz/Aeff
                         - (signal3+s3k2/2)*fiber_loss - (signal3+s3k2/2)*stokes3_loss)*delta_L
        s4k3 = (signal3*(signal4+s4k2/2)*raman_gain + signal3*raman_gain*hnu_signal4*raman_width_Hz/Aeff
                         - (signal4+s4k2/2)*fiber_loss - (signal4+s4k2/2)*stokes4_loss)*delta_L
        
        fk4 = (-(fundamental+fk3)*signal1*raman_gain*stokesw1/fundamental_wavelength
                              - (fundamental+fk3)*fiber_loss)*delta_L
        fbk4 = (-(fund_back+fbk3)*signal1*raman_gain*stokesw1/fundamental_wavelength
                              - (fundamental+fk3)*fiber_loss)*delta_L
        s1k4 = ((fundamental+fund_back)*(signal1+s1k3)*raman_gain
                        -(signal1+s1k3)*signal2*raman_gain*stokesw2/stokesw1
                        -(signal1+s1k3)*fiber_loss)*delta_L
        s2k4 = (signal1*(signal2+s2k3)*raman_gain + signal1*raman_gain*hnu_signal2*raman_width_Hz/Aeff
                         - (signal2+s2k3)*fiber_loss - (signal2+s2k3)*stokes2_loss)*delta_L
        s3k4 = (signal2*(signal3+s3k3)*raman_gain + signal2*raman_gain*hnu_signal3*raman_width_Hz/Aeff
                         - (signal3+s3k3)*fiber_loss - (signal3+s3k3)*stokes3_loss)*delta_L
        s4k4 = (signal3*(signal4+s4k3)*raman_gain + signal3*raman_gain*hnu_signal4*raman_width_Hz/Aeff
                         - (signal4+s4k3)*fiber_loss - (signal4+s4k3)*stokes4_loss)*delta_L

        y=[fundamental,fund_back,signal1,signal2,signal3,signal4]
        fundamental = fundamental + (fk1/6+fk2/3+fk3/3+fk4/6)*int_mask
        fund_back = fund_back + (fbk1/6+fbk2/3+fbk3/3+fbk4/6)*int_mask
        signal1 = signal1 + (s1k1/6+s1k2/3+s1k3/3+s1k4/6)*int_mask
        signal1 = convolve(convsignal,signal1,mode='same')
        signal2 = signal2 + (s2k1/6+s2k2/3+s2k3/3+s2k4/6)*int_mask
        signal2 = convolve(convsignal,signal2,mode='same')    
        signal3 = signal3 + (s3k1/6+s3k2/3+s3k3/3+s3k4/6)*int_mask
        signal3 = convolve(convsignal,signal3,mode='same')
        signal4 = signal4 + (s4k1/6+s4k2/3+s4k3/3+s4k4/6)*int_mask
        signal4 = convolve(convsignal,signal4,mode='same')
        
        signal1 = np.roll(signal1,fact)
        signal2 = np.roll(signal2,fact)
        signal3 = np.roll(signal3,fact)
        signal4 = np.roll(signal4,fact)
        fundamental = np.roll(fundamental,1*fact)
        fund_back = np.roll(fund_back,-1*fact)

        P.value=i

        if i % int(N/(2*MM+1)/M/10) == 0 or i == int(round(N/(2*MM+1)/M))-1:
            cff.data_source.data['x']=t
            cff.data_source.data['y']=fundamental*Aeff*np.heaviside(tend3-t,0)
            cfb.data_source.data['y']=fund_back*Aeff*np.heaviside(tend3-t,0)
            c1.data_source.data['y']=signal1*Aeff
            c2.data_source.data['y']=signal2*Aeff
            c3.data_source.data['y']=signal3*Aeff
            c4.data_source.data['y']=signal4*Aeff
            push_notebook(handle=nh)
    push_notebook(handle=nh)

    Eps[0,k+1]=np.sum(signal1)*Aeff*delta_t*1e3
    Eps[1,k+1]=np.sum(signal2)*Aeff*delta_t*1e3
    Eps[2,k+1]=np.sum(signal3)*Aeff*delta_t*1e3
    Eps[3,k+1]=np.sum(signal4)*Aeff*delta_t*1e3
    signal2 = signal2 * stokes2_loss_lump
    P2.value=k+1
pg.add_layout(label1)
pg.add_layout(label1a)
pg.add_layout(label2)
pg.add_layout(label2a)
pg.add_layout(label3)
pg.add_layout(label3a)
pg.add_layout(label4)
pg.add_layout(label4a)
label1a.text='= {} mJ'.format("%2.3f" % Eps[0,-1])
label2a.text='= {} mJ'.format("%2.3f" % Eps[1,-1])
label3a.text='= {} mJ'.format("%2.3f" % Eps[2,-1])
label4a.text='= {} mJ'.format("%2.3f" % Eps[3,-1])

push_notebook(handle=nh)

"""
np.save('fundamental_1.npy', fundamental)
np.save('signal1_1.npy', signal1)
np.save('signal2_1.npy', signal2)
np.save('Aeff_1.npy', Aeff)
"""

t1=time.time()
print('elapsed time=',t1-t0, 'seconds')

IntProgress(value=0, bar_style='success', description='Progress', max=1001)

IntProgress(value=0, bar_style='success', description='Progress', max=1)

In [ ]:
cs.data_source.data['x'].shape

In [ ]:
z.shape

In [47]:
np.sum(convsignal)

250.62857803533558

In [39]:
t

array([-1.30089997e-09, -1.29989983e-09, -1.29889970e-09, ...,
        2.29959198e-09,  2.30059212e-09,  2.30159226e-09])

In [40]:
FWHM1

1e-10

In [54]:
np.max(convsignal)

0.003989952577504013

In [50]:
signal1

array([0., 0., 0., ..., 0., 0., 0.])

In [52]:
np.max(signal1)

0.0